# MariaDB with Python Exercises

Make sure that you finished the Basic SQL Exercises before starting this exercise because you will be converting what you have already done from manual inputs to using Python and MariaDB's Python connector to interface directly with MariaDBMS. You are still encouraged to use MySQL Client to help check your work.

---

**Q1)** Write SQL statements to answer the questions using the following table.

Table name: salesman

| salesman_id |    name    |   city   | commission |
|------------:|:-----------|:---------|-----------:|
|        5001 | James Hoog | New York |       0.15 |
|        5002 | Nail Knite | Paris    |       0.13 |
|        5005 | Pit Alex   | London   |       0.11 |
|        5006 | Mc Lyon    | Paris    |       0.14 |
|        5007 | Paul Adam  | Rome     |       0.13 |
|        5003 | Lauson Hen | San Jose |       0.12 |
        
1. Return all the salesman data for salesman that either comes from the city Paris or London.
2. Find the average commission for salesmen from Paris.
3. Find the salesman ids of those salesmen with names starting with the letter "P".

**Answers**

**Qn 1:** SELECT * FROM salesman WHERE city = "Paris" OR city = "London";

**Qn 2:** SELECT AVG(commission) FROM salesman WHERE city = "Paris";
*or* SELECT AVG(commission) avg_comm FROM salesman WHERE city = "Paris";

**Qn 3:** SELECT salesman_id FROM salesman WHERE name LIKE "P%";

---
**Q2)** Write SQL statements to answer the questions using the following table. 

Table name: salesman

| salesman_id |    name    |   city   | commission | 
|------------:|:-----------|:---------|-----------:|
|        5001 | James Hoog | New York |       0.15 |
|        5002 | Nail Knite | Paris    |       0.13 |
|        5005 | Pit Alex   | London   |       0.11 |
|        5006 | Mc Lyon    | Paris    |       0.14 |
|        5007 | Paul Adam  | Rome     |       0.13 |
|        5003 | Lauson Hen | San Jose |       0.12 |
        
Table name: customer

| cust_id     |   cust_name    |    city    | grade | salesman_id |
|------------:|:---------------|:-----------|------:|------------:|
|        3002 | Nick Rimando   | New York   |   100 |        5001 |
|        3007 | Brad Davis     | New York   |   200 |        5001 |
|        3005 | Graham Zusi    | California |   200 |        5002 |
|        3008 | Julian Green   | London     |   300 |        5002 |
|        3004 | Fabian Johnson | Paris      |   300 |        5006 |
|        3009 | Geoff Cameron  | Berlin     |   100 |        5003 |
|        3003 | Jozy Altidor   | Moscow     |   200 |        5007 |
|        3001 | Brad Guzan     | London     |   250 |        5005 |
        
1. Return both the customer's and salesmen's name and id who lives in the same city.
2. Return the salesman's name and id where customers have graded them over 250.
3. Customer Brad Davis have moved from New York to Hong Kong, update the customer's data to reflect this change.
4. Using the following data, insert the following customer records:
 <pre>
 {'new_customers':
     ({'cust_id': 3006, 'cust_name': 'Joe Black', 'city': 'California', 'grade': 200, 'attendant_salesman':'Pit Alex'},
      {'cust_id': 3010, 'cust_name': 'Blue Lightning', 'city': 'Hamburg', 'grade': 300, 'attendant_salesman':'Paul Adam'},
      {'cust_id': 3011, 'cust_name': 'Lyra Slivertongue', 'city': 'Oxford', 'grade': 300, 'attendant_salesman':'Nail Knite'}
     )}
 </pre>
 you are to use the dictionary data as it is.

**Answers**

**Qn 1:** SELECT c.cust_name, c.cust_id, s.name, s.salesman_id FROM customer c, salesman s WHERE s.city = c.city;

**Qn 2:** SELECT s.salesman_id, s.name FROM customer c, salesman s WHERE s.salesman_id = c.salesman_id AND c.grade > 250;

**Qn 3:** UPDATE customer SET city = "Hong Kong" WHERE cust_name = "Brad Davis";

**Qn 4:** INSERT INTO customer SELECT 3006, 'Joe Black', 'California', 200, s.salesman_id FROM salesman AS s WHERE s.name = 'Pit Alex';

In [ ]:
from mariadb_context import MariaDBCursor

data = {'new_customers':
  ({'cust_id': 3006, 'cust_name': 'Joe Black', 'city': 'California', 'grade': 200, 'attendant_salesman':'Pit Alex'},
   {'cust_id': 3010, 'cust_name': 'Blue Lightning', 'city': 'Hamburg', 'grade': 300, 'attendant_salesman':'Paul Adam'},
   {'cust_id': 3011, 'cust_name': 'Lyra Slivertongue', 'city': 'Oxford', 'grade': 300, 'attendant_salesman':'Nail Knite'}
  )}

insert_sql = []
for cust in data['new_customers']:
    # method 1
    sql = f'INSERT INTO customer SELECT {cust["cust_id"]}, \'{cust["cust_name"]}\', \'{cust["city"]}\', {cust["grade"]}, s.salesman_id ' + \
          f'FROM salesman AS s WHERE s.name = \'{cust["attendant_salesman"]}\';'
    
    # method 2
#     sql = 'INSERT INTO customer SELECT {cust_id}, \'{cust_name}\', \'{city}\', {grade}, s.salesman_id '.format(**cust) + \
#           'FROM salesman AS s WHERE s.name = \'{attendant_salesman}\';'.format(**cust)
    
    insert_sql.append(sql)
insert_sql


with MariaDBCursor(<your mariaDB password>, <table name>) as rdbms:
    for sql in insert_sql:
        rdbms.execute(sql)

---
**Q3)** Given the following database schema (from lecture notes) below, answer the following questions

**Student**(sID CHAR(8), sName CHAR(50), gender CHAR(1), age INT, dID CHAR(2), grade CHAR(2))    
**Dept**(dID CHAR(2), dName CHAR(20), dean CHAR(50))   
**Course**(cID CHAR(3), cName CHAR(50), hours INT, credit INT, iID CHAR(3))   
**Instructor**(iID CHAR(3), iName CHAR(50), dID CHAR(2), workload FLOAT)   
**RC**(sID CHAR(8), cID CHAR(3), score FLOAT)    

1. Find the names of the students who are taking the courses taught by "Lim".
2. Using the function aggregate function `MAX()`, find the oldest female student in the CS department.
3. Using the not equal to operator (`<>`) and the `ORDER BY` clause, find all the students who are not from the CS department and group them by department.
4. Using the `GROUP BY` clause, return the avg age of the students for each department.

**Answers**

**Qn 1:** SELECT s.sName FROM Student s, RC rc, Course c, Instructor i WHERE s.sID = rc.sID and rc.cID = c.cID and i.iID = c.iID and i.iName LIKE "Lim";

**Qn 2:** SELECT MAX(s.age) max_female_student FROM Student s WHERE s.dID = "CS" and s.gender = "F";

**Qn 3:** SELECT * FROM Student s WHERE s.dID <> "CS" ORDER BY s.dID;

**Qn 4:** SELECT dID, AVG(age) FROM Student GROUP BY dID;